<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math

Загрузим данные и проведем предобраотку данных как на семинаре.

In [3]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open articles-sharing-reading-from-cit-deskdrop.zip, articles-sharing-reading-from-cit-deskdrop.zip.zip or articles-sharing-reading-from-cit-deskdrop.zip.ZIP.


In [2]:
!cd ./data && gdown 'https://www.dropbox.com/s/z8syrl5trawxs0n/articles.zip?dl=0' && unzip articles.zip && rm articles.zip

/bin/bash: line 0: cd: ./data: No such file or directory


Не получилось загрузить данные по ссылке, скачал руками и вставил в колаб

In [2]:
articles_df = pd.read_csv("shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [3]:
interactions_df = pd.read_csv("users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [4]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [5]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [6]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [7]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [8]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [9]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [10]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [11]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

ValueError: ignored

# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [14]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [22]:
# Ваш код здесь
from scipy.sparse import csr_matrix

def razrezh_matr(interactions_df, users,
                         contents):    
    matr = pd.DataFrame(0, columns=contents, index=users)
    
    user_ids = interactions_df['personId'].values
    content_ids = interactions_df['contentId'].values
    eventStrengths = interactions_df['eventStrength'].values
    
    for i in range(len(interactions_df)):
        matr.loc[user_ids[i], content_ids[i]] = eventStrengths[i] if eventStrengths[i] is not np.nan else 0
    return csr_matrix(matr.values)

total_content_ids = set(interactions_train_df['contentId']) | set(interactions_test_df['contentId'])
total_users_ids = set(interactions_train_df['personId']) | set(interactions_test_df['personId'])

train = razrezh_matr(interactions_train_df, total_users_ids, total_content_ids)
test = razrezh_matr(interactions_test_df, total_users_ids, total_content_ids)

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [18]:
model = LightFM(loss='warp')
model.fit(train, epochs=20)

answer_2 = precision_at_k(model, test, k=10, train_interactions=train).mean()

print(answer_2)


0.006211813


## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [19]:
# Ваш код здесь
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform(articles_df['text'])

model.fit(train, item_features=feat, epochs=20)

answer_3 = precision_at_k(model, test, 
                                item_features=feat, 
                                k=10, train_interactions=train).mean()

print(answer_3)


0.0050916495


Стало хуже

## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [20]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer


stop_words = stopwords.words('english') #стоп-слова в английском
porter_stemmer = PorterStemmer() #Перевод в начальную форму слова

def stemming_tokenizer(str_input):    
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()    
    
    words = [porter_stemmer.stem(word)
             for word in words 
             if word not in stop_words]
    return words

vectorizer = TfidfVectorizer(analyzer='word', tokenizer=stemming_tokenizer)
feat = vectorizer.fit_transform(articles_df['text'])

model.fit(train, item_features=feat, epochs=20)

answer_4 = precision_at_k(model, test,
                          item_features=feat, 
                          k=10, train_interactions=train).mean()

print(answer_4)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
0.007026477


Улучшилось ли качество предсказания?

Да, precision стал выше, следовательно качество модели улучшилось

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [25]:
from bayes_opt import BayesianOptimization

parameters = {
    'components_num': (20, 80),
    'epoch_num': (5, 50)
}


def optimisation_func(components_num, epoch_num):
    model = LightFM(learning_rate=0.05, loss='warp',
                    no_components=int(components_num))
    model.fit(train, item_features=feat, epochs=int(epoch_num))

    precision = precision_at_k(model, test, item_features=feat, k=10,
                               train_interactions=train).mean()
    
    return precision
  
optimizer = BayesianOptimization(
  f = optimisation_func,
  pbounds = parameters,
  verbose = 5,
  random_state = 5, 
 )

optimizer.maximize(
  init_points = 4,
  n_iter = 3, 
 )

|   iter    |  target   | compon... | epoch_num |
-------------------------------------------------
|  1        |  0.008045 |  33.32    |  44.18    |
|  2        |  0.007332 |  32.4     |  46.34    |
|  3        |  0.007943 |  49.3     |  32.53    |
|  4        |  0.008554 |  65.95    |  28.33    |
|  5        |  0.007637 |  46.55    |  29.83    |
|  6        |  0.006823 |  21.08    |  9.468    |
|  7        |  0.006823 |  33.12    |  44.31    |


NameError: ignored

In [26]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=32)
model.fit(train, item_features=feat, epochs=46)

test_precision = precision_at_k(model, test, item_features=feat,
                                k=10, train_interactions=train).mean()
print(test_precision)

0.0065173116


Получше чем стандартная модель, но похуже, чем модель с предобработанным текстом

## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь